Code to experiment with calculating calibration.

In [1]:
import pandas as pd

Get topics for news the user clicked on and news user was shown (impressions)

In [32]:
behavior_df = pd.read_table("../data/test_mind_large/behaviors.tsv",
               header='infer',
               usecols=range(5),
               names=[
                   'impression_id', 'user', 'time',
                   'clicked_news', 'impressions'
               ])
behavior_df.fillna('', inplace = True)
behavior_df.head()
# Users may appear multiple times in behavior_df. 
# For the purposes of this experiment, we are treating each row as unique and 
# extracting history and the current impressions. However in the future, 
# all history for a user may need to be considered.

In [39]:
news_df = pd.read_table("../data/test_mind_large/news.tsv",
            header = 'infer', 
            usecols = range(4),
            names = ['news_id', 'topic', 'subtopic', 'title'])
news_df.head()

,news_id,topic,subtopic,title
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an..."
1,N23144,health,weightloss,50 Worst Habits For Belly Fat
2,N86255,health,medical,Dispose of unwanted prescription drugs during ...
3,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...
4,N75236,health,voices,I Was An NBA Wife. Here's How It Affected My M...


In [50]:
user_clicked_topics = behavior_df[['impression_id', 'clicked_news']]
user_clicked_topics['news_id'] = user_clicked_topics['clicked_news'].apply(lambda x: x.split(' '))
user_clicked_topics = user_clicked_topics.explode('news_id')
user_clicked_topics.drop(columns=['clicked_news'], inplace=True)
user_clicked_topics = pd.merge(user_clicked_topics, news_df)
user_clicked_topics = user_clicked_topics.groupby(by='impression_id')['topic'].apply(list).reset_index(name="clicked_topics")

C:\Users\Eliza\AppData\Local\Temp\ipykernel_13948\1990249385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_clicked_topics['news_id'] = user_clicked_topics['clicked_news'].apply(lambda x: x.split(' '))


In [53]:
user_shown_topics = behavior_df[['impression_id', 'impressions']]
user_shown_topics['news_id'] = user_shown_topics['impressions'].apply(lambda x: x.split(' '))
user_shown_topics = user_shown_topics.explode('news_id')
user_shown_topics['news_id'] = user_shown_topics['news_id'].apply(lambda x: x.split("-")[0])
user_shown_topics.drop(columns=['impressions'], inplace=True)
user_shown_topics = pd.merge(user_shown_topics, news_df)
user_shown_topics = user_shown_topics.groupby(by='impression_id')['topic'].apply(list).reset_index(name="shown_topics")

C:\Users\Eliza\AppData\Local\Temp\ipykernel_13948\4280619239.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_shown_topics['news_id'] = user_shown_topics['impressions'].apply(lambda x: x.split(' '))


In [54]:
topics_df = pd.merge(user_clicked_topics, user_shown_topics)

In [55]:
topics_df

,impression_id,clicked_topics,shown_topics
0,1,"[foodanddrink, health, news, travel, sports, l...","[movies, lifestyle, news, tv, lifestyle, fooda..."
1,2,"[tv, tv, sports, news, sports, news, entertain...","[lifestyle, autos, news, travel, video, entert..."
2,3,"[sports, movies, lifestyle, sports, travel, ne...","[lifestyle, sports, movies, news, sports, spor..."
3,4,"[foodanddrink, lifestyle, foodanddrink, tv, ne...","[foodanddrink, lifestyle, weather, autos, vide..."
4,5,"[news, travel, news, news, movies, tv, news, l...","[lifestyle, health, lifestyle, foodanddrink, s..."
...,...,...,...
365196,376467,"[lifestyle, lifestyle, travel, entertainment, ...","[sports, sports, news, lifestyle]"
365197,376468,"[news, news, finance, weather, sports, sports,...","[sports, lifestyle, weather, lifestyle, lifest..."
365198,376469,"[movies, news, sports, sports, tv, sports, new...","[news, news, music, weather, news, news, finan..."
365199,376470,"[news, autos, health, tv, finance, tv, sports,...","[news, news, tv, lifestyle, finance, sports, m..."


Calculate calibration